In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import fitz 

C:\Users\ACER\AppData\Local\Temp\ipykernel_22004\2069252543.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
all_data =[]
count = 0
num = 1000 

In [6]:
for i in range(2):

    url=f'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-computer_science=y&classification-physics_archives=all&classification-include_cross_list=include&date-filter_by=all_dates&date-year=&date-from_date=&date-to_date=&date-date_type=submitted_date&abstracts=show&size=50&order=-announced_date_first&start={str(num)}'
    print(url+'\n\n\n\n\n')
    response = requests.get(url)
    response = response.content

    soup = BeautifulSoup(response,'html.parser')

    papers = soup.find_all('li', class_='arxiv-result')
    for i in range(len(papers)):
        title = papers[i].find('p',class_='title')
        authors = papers[i].find_all('p',class_='authors')
        noob_authors = []
        single = ''
        for author in authors:
            author_names = author.find_all('a')
            for author_name in author_names:
                noob_authors.append(author_name.text)
        abstracts = papers[i].find('span',class_='abstract-full')
        single = ', '.join(noob_authors)
        
        pdf_link = papers[i].find('span')
        pdf_link = pdf_link.find('a')
        noob_data = {'title':'','authors':'','abstract':'','pdf_link':'','textdata':''}
        noob_data['title'] = title.text.strip()
        noob_data['authors'] = single
        noob_data['abstract'] = abstracts.text
        # if noob_data['pdf_link']:
        try:
            noob_data['pdf_link'] = pdf_link['href']
        except:
            noob_data['pdf_link'] = 'No link'
            
        print(f"Article {count} extracted")
        count +=1
        # print(noob_data)
        all_data.append(noob_data)
        
    num+=50

https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-computer_science=y&classification-physics_archives=all&classification-include_cross_list=include&date-filter_by=all_dates&date-year=&date-from_date=&date-to_date=&date-date_type=submitted_date&abstracts=show&size=50&order=-announced_date_first&start=1000





Article 0 extracted
Article 1 extracted
Article 2 extracted
Article 3 extracted
Article 4 extracted
Article 5 extracted
Article 6 extracted
Article 7 extracted
Article 8 extracted
Article 9 extracted
Article 10 extracted
Article 11 extracted
Article 12 extracted
Article 13 extracted
Article 14 extracted
Article 15 extracted
Article 16 extracted
Article 17 extracted
Article 18 extracted
Article 19 extracted
Article 20 extracted
Article 21 extracted
Article 22 extracted
Article 23 extracted
Article 24 extracted
Article 25 extracted
Article 26 extracted
Article 27 extracted
Article 28 extracted
Article 29 extracted
Articl

In [7]:
df = pd.DataFrame().from_dict(all_data)

In [8]:
count = 0
# textdata = []
for item in all_data:
    count+=1
    if count == 100:
        break
    pdf_link = item['pdf_link']
    response = requests.get(pdf_link)
    response.raise_for_status()
    pdf_document = fitz.open(stream=response.content, filetype="pdf")
            
    # Initialize an empty string to store text
    text = ''
    textdict = {'textData':''}
    # Iterate over pages and extract text
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()
    all_data[count-1]['textdata'] = text
    # noob_data['textdata'][count-1]= text
    # textdata.append(textdict)
    print(f"{count}. Text Extracted from {item['title']}")
    # print(text)
    # Close the PDF file
    pdf_document.close()

1. Text Extracted from Text Region Multiple Information Perception Network for Scene Text Detection
2. Text Extracted from Gender Bias in Machine Translation and The Era of Large Language Models
3. Text Extracted from Towards Hierarchical Spoken Language Dysfluency Modeling
4. Text Extracted from Optimizing Medication Decisions for Patients with Atrial Fibrillation through Path Development Network
5. Text Extracted from CPCL: Cross-Modal Prototypical Contrastive Learning for Weakly Supervised Text-based Person Re-Identification
6. Text Extracted from An optimization-based equilibrium measure describes non-equilibrium steady state dynamics: application to edge of chaos
7. Text Extracted from Attack tree metrics are operad algebras
8. Text Extracted from Spintronic logic: from transducers to logic gates and circuits
9. Text Extracted from Advancing Large Multi-modal Models with Explicit Chain-of-Reasoning and Visual Question Generation
10. Text Extracted from Distantly Supervised Morpho-

In [14]:
df = pd.DataFrame().from_dict(all_data)

In [25]:
#check no link
df['title'] = df['title'].astype(str)
df['title' == df['title'].isin(['No link'])]

,title,authors,abstract,pdf_link,textdata


In [18]:
df.to_csv('dataset(1100-1200).csv')